In [ ]:
!pip install textblob

In [ ]:
!pip install --user -U nltk

In [ ]:
import os 
import pandas as pd
dir_name= os.getcwd()+'/sentiment-data/'


hkex_files=os.path.join(dir_name,'stock_ticker_datasets/hkex.csv')

hkex=pd.read_csv(hkex_files) 
# hkex[hkex.iloc['Category'] == 'Equity Securities']

hkex=hkex.loc[hkex['Category'] == 'Equity']

hkex['Ticker']=hkex['Ticker'].astype(str)
hkex_input=hkex['Ticker']


n = 500  #chunk row size
# 2554
hkex_df = [hkex_input[i:i+n] for i in range(0,len(hkex_input),n)]
# test_df=['669','175','1211','2319','6186','168','2269','6618','01801','1','4','19','700','3690','9988','823','16','1113','1299','5','939','2','3','2688','941','762','6823','968','868','2382','2899','1818','2689','883','386','857']
# print(len(test_df))
# hkex.set_index("Ticker" , inplace=True)
print(hkex_df)


In [ ]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import twitter_samples 

nltk.downloader.download('vader_lexicon')


In [ ]:
analyser = SentimentIntensityAnalyzer()

In [ ]:
#### Function for Vader Analysis 

# read from vader scores
def read_news_vader_path(df):
    print('read in datasets')
    cs=[]
    # append a compound score to every news row
    for row in range(len(df)):
        cs.append(analyser.polarity_scores(df['news'].iloc[row])['compound'])
    # append the column to original dataset
    df['compound_vader_score']=cs
    return df


# group by the mean compound vader score by dates
def find_news_vader_pred_label(df,threshold):
    print('find_pred_label')
    news = df['news']
    # group the data by dates
    df = df.groupby(['dates'])['compound_vader_score'].mean().reset_index()
    final_label=[]
    
    # convert the vader score using a threshold to a sentiment label
    for i in range(len(df)):

        if df['compound_vader_score'].iloc[i] > threshold:
            final_label.append(2)
        elif df['compound_vader_score'].iloc[i] < -threshold:
            final_label.append(0)
        elif (df['compound_vader_score'].iloc[i] >= -threshold  
              and df['compound_vader_score'].iloc[i] <= threshold):
            final_label.append(1)

    df['vader_label'] = final_label
    return df


# merge the dataset with the hang seng index daily moving average
def merge_vader_actual_label (df,hsi_movement_df):
    print('merge_actual_label')
    vader_data = df
    vader_data.set_index(keys = ["dates"],inplace=True)
    label_data = pd.read_csv(hsi_movement_df)
    label_data.set_index(keys = ["dates"],inplace=True)
    # inner join the two datasets using the date index
    merge = pd.merge(vader_data,label_data, how='inner', left_index=True, right_index=True)
    merge = merge.reset_index()
    # drop the redudant column 
    merge = merge.drop(['Unnamed: 0'],axis=1)
    
    return merge

In [ ]:
#### Function for Textblob Analysis 


from textblob import TextBlob
# append the normalize textblob (-1 to 1) score to the corresponding news
def read_news_textblob_path(df):
    print('read in datasets')
    cs=[]
    # append a compound score to every news row
    for row in range(len(df)):
        cs.append(TextBlob(df['news'].iloc[row]).sentiment[0])
    # append the column to original dataset
    df['compound_textblob_score']=cs
#     print(df)
    return df


# group by the mean compound textblob score by dates
def find_news_textblob_pred_label(df,threshold):
    print('find_pred_label')
#     print(df)
    news = df['news']
    # group the data by dates
    df = df.groupby(['dates'])['compound_textblob_score'].mean().reset_index()
    final_label=[]
    
    # convert the vader score using a threshold to a sentiment label
    for i in range(len(df)):

        if df['compound_textblob_score'].iloc[i] > threshold:
            final_label.append(2)
        elif df['compound_textblob_score'].iloc[i] < -threshold:
            final_label.append(0)
        elif (df['compound_textblob_score'].iloc[i] >= -threshold  
              and df['compound_textblob_score'].iloc[i] <= threshold):
            final_label.append(1)

    df['textblob_label'] = final_label
    return df

# merge the dataset with the hang seng index daily moving average
def merge_textblob_actual_label (df,hsi_movement_df):
    print('merge_actual_label')
    textblob_data = df
    textblob_data.set_index(keys = ["dates"],inplace=True)
    label_data = pd.read_csv(hsi_movement_df)
    label_data.set_index(keys = ["dates"],inplace=True)
    # inner join the two datasets using the date index
    merge = pd.merge(textblob_data,label_data, how='inner', left_index=True, right_index=True)
    merge = merge.reset_index()
    # drop the redudant column 
    merge = merge.drop(['Unnamed: 0'],axis=1)
    
    return merge





In [ ]:
import pandas as pd
from nltk.tokenize import WordPunctTokenizer as wpt
import math


def build_dict():
    
    #Load master dictionary
    path = os.path.join(dir_name,'train-data/hkex/LoughranMcDonald_MasterDictionary_2016.csv') 
    
    master = pd.read_csv(path)
#     print(master)
    positive = master[master['Positive']>0]
#     print('positive %s', positive)
    negative = master[master['Negative']>0]
#     print('negative %s' negative)
    pos_words = positive['Word'].tolist()
    neg_words = negative['Word'].tolist()
    
    pos_words = [word.lower() for word in pos_words]
    neg_words = [word.lower() for word in neg_words]
    stopwords = pd.read_csv("https://drive.google.com/file/d/0B4niqV00F3msSktONVhfaElXeEk/view?usp=sharing",names=['Word'])
    stop_list = stopwords['Word'].tolist()
    return pos_words,neg_words,stop_list


def process_news_Loughran(df,pos_words,neg_words,stop_lists):
    print('read in datasets')
    cs=[]
    
    # tokenize the words in each news
    for row in range(len(df)):
        cs.append(word.lower() for word in wpt().tokenize(df['news'].iloc[row]))

#     Remove the stopwords
    words_new = [word for word in cs if word not in stop_lists]

    Loughran_label=[]
    for words_new in cs:
       
        words_new_pos = [word for word in words_new if word in pos_words]
        words_new_neg = [word for word in words_new if word in neg_words]
        
        if(len(words_new_pos)>len(words_new_neg)):
            Loughran_label.append(2)
        elif(len(words_new_pos)<len(words_new_neg)):
            Loughran_label.append(0)
        else:
            Loughran_label.append(1)
        print(Loughran_label)
        
    df['loughran_label']=Loughran_label
    df = df.groupby(['dates'])['loughran_label'].mean().round().reset_index()
#     print(df)
    return df

# merge the dataset with the hang seng index daily moving average
def merge_textblob_actual_label (df,hsi_movement_df):
    print('merge_actual_label')

    df.set_index(keys = ["dates"],inplace=True)
    label_data = pd.read_csv(hsi_movement_df)
    label_data.set_index(keys = ["dates"],inplace=True)
    # inner join the two datasets using the date index
    merge = pd.merge(df,label_data, how='inner', left_index=True, right_index=True)
    merge = merge.reset_index()
    # drop the redudant column 
    merge = merge.drop(['Unnamed: 0'],axis=1)
    
    return merge


In [ ]:
### Starter function for vader sentiment analysis ###

def starter_vader(ticker):
    # get the full path of each ticker
    path = os.path.join(dir_name,'data-news/data-aastock-equities/'+'data-'+ticker.zfill(5)+'-aastock.csv')           
    df = pd.read_csv(path,names=['dates','news','ticker','newstype'])
    # read append the compound vader score to the pandas dataframe
    df = read_news_vader_path(df)
    # pass in the threshold to get the vader label
    df = find_news_vader_pred_label(df,0.01)
            
    result_path = os.path.join(dir_name,'data-results/hkex-aastock/'+'data-'+ticker.zfill(5)+'-result.csv')
    
    # get the full path of the hang seng index average csv file
    hsi_movement_path = os.path.join(dir_name,'train-data/hkex/hsi_movement.csv')  
    # merge the df pandas with the hsi_average
    df = merge_vader_actual_label (df,hsi_movement_path)
    print(df)
    # store to the csv file if the dataset is not empty
    if (df.empty == False):
        df.to_csv(result_path,index=False)

# starter_vader('1')

In [ ]:
### Starter function for textblolb sentiment analysis ###

def starter_textblob(ticker):
    # get the full path of each ticker
    path = os.path.join(dir_name,'data-news/data-aastock-equities/'+'data-'+ticker.zfill(5)+'-aastock.csv')           
    df = pd.read_csv(path,names=['dates','news','ticker','newstype'])
    # read append the compound vader score to the pandas dataframe
    df = read_news_textblob_path(df)
    # pass in the threshold to get the vader label
    df = find_news_textblob_pred_label(df,0.01)
            
    result_path = os.path.join(dir_name,'data-results/hkex-aastock/'+'data-'+ticker.zfill(5)+'-result.csv')
    
    # get the full path of the hang seng index average csv file
    hsi_movement_path = os.path.join(dir_name,'train-data/hkex/hsi_movement.csv') 
    db_df=pd.read_csv(result_path)
    # merge the df pandas with the hsi_average
    df = merge_textblob_actual_label (df,hsi_movement_path)
#     print(df)
#     print(db_df)
    db_df['compound_textblob_score']=df['compound_textblob_score']
    db_df['textblob_label']=df['textblob_label']
    
    # store to the csv file if the dataset is not empty
    print(db_df)
    if (db_df.empty == False):
        db_df.to_csv(result_path,index=False)


        


In [ ]:
### Starter function for Loughran dictionary sentiment analysis ###

def starter_Loughran(ticker):
    # get the full path of each ticker
    path = os.path.join(dir_name,'data-news/data-aastock-equities/'+'data-'+ticker.zfill(5)+'-aastock.csv')           
    df = pd.read_csv(path,names=['dates','news','ticker','newstype'])
    # 
   
    pos_words,neg_words,stop_lists= build_dict()
    df = process_news_Loughran(df,pos_words,neg_words,stop_lists)
    
    result_path = os.path.join(dir_name,'data-results/hkex-aastock/'+'data-'+ticker.zfill(5)+'-result.csv')
    
    # get the full path of the hang seng index average csv file
    hsi_movement_path = os.path.join(dir_name,'train-data/hkex/hsi_movement.csv') 
    db_df=pd.read_csv(result_path)
    
    df = merge_textblob_actual_label (df,hsi_movement_path)
    db_df['loughran_label']=df['loughran_label']

    
#     print(db_df)
    
    # store to the csv file if the dataset is not empty
    if (db_df.empty == False):
        db_df.to_csv(result_path,index=False)
# starter_Loughran('1')

In [ ]:
# starter_vader('1') 
# starter_textblob('1')

# collect sentiment label for tickers in hkex    
for tickers in hkex_df:
     for ticker in tickers:
            try:
                print(ticker)
#                 starter_vader(ticker)  
#                 starter_textblob(ticker)  
                starter_Loughran(ticker)
            except Exception as e:
                print(e)
                pass
                
            
